## 图书分类

 导入相关的包 


In [1]:
import sys
sys.path.append("/usr/local/anaconda2/envs/pt-tf-env/lib/python3.6/site-packages")

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn import metrics
import joblib
# from bayes_opt import BayesianOptimization
from gensim import models

#### Part 1: 读取训练好的词向量和训练/测试数据集
#### 在运行此文件之前，需要先运行embeddin文件生成词向量文件。

In [3]:
max_length = 500  # 表示样本表示最大的长度,表示降维之后的维度
sentence_max_length = 1500  # 表示句子/样本在降维之前的维度
Train_features3, Test_features3, Train_label3, Test_label3 = [], [], [], []
# 加载预训练好的embedding
fast_embedding = models.KeyedVectors.load('fast_model')
w2v_embedding = models.KeyedVectors.load('w2v_model')

print("fast_embedding输出词表的个数{},w2v_embedding输出词表的个数{}".format(
    len(fast_embedding.wv.vocab.keys()), len(w2v_embedding.wv.vocab.keys())))

print("取词向量成功")

train = pd.read_csv('data/train_clean.csv', sep='\t')
#dev = pd.read_csv('dev_clean.tsv', sep='\t')
test = pd.read_csv('data/test_clean.csv', sep='\t')
print("读取数据完成")


fast_embedding输出词表的个数19592,w2v_embedding输出词表的个数19592
取词向量成功
读取数据完成


 将df中的label映射为数字标签并保存到labelIndex列中

In [4]:
labelName = train.label.unique()  # 全部label列表
labelIndex = list(range(len(labelName)))  # 全部label标签
labelNameToIndex = dict(zip(labelName, labelIndex))  # label的名字对应标签的字典
labelIndexToName = dict(zip(labelIndex, labelName))  # label的标签对应名字的字典
train["labelIndex"] = train.label.map(labelNameToIndex)
test["labelIndex"] = test.label.map(labelNameToIndex)

In [5]:
print(test["labelIndex"])

0         1
1         1
2         1
3         6
4         1
         ..
29469    11
29470     1
29471    21
29472     1
29473     5
Name: labelIndex, Length: 29474, dtype: int64


In [6]:
def query_cut(query):
    '''
    该函数用于对输入的语句（query）按照空格进行切分
    '''
    return query.split(' ')


train["queryCut"] = train["text"].apply(query_cut)
# dev["queryCut"] = dev["text"].apply(query_cut)
test["queryCut"] = test["text"].apply(query_cut)
print("切分数据完成")

切分数据完成


 读取停用词文件 并去除样本中的停用词


In [7]:
with open('data/stopwords.txt', "r") as f:
    stopWords = f.read().split("\n")
def rm_stop_word(wordList):
    return [word for word in wordList if word not in stopWords]

train["queryCutRMStopWord"] = train["queryCut"].apply(rm_stop_word)
# dev["queryCutRMStopWord"] = dev["text"].apply(rm_stop_word)
test["queryCutRMStopWord"] = test["queryCut"].apply(rm_stop_word)
print("去除停用词")
print(type(train["queryCutRMStopWord"]))

去除停用词
<class 'pandas.core.series.Series'>


In [9]:
def Find_embedding_with_windows(embedding_matrix):
    '''
    函数说明：该函数用于获取在不同滑动窗口下（size=2,3,4）经过卷积池化操作之后拼接而成的词向量
    参数说明：
        - embedding_matrix：样本中所有词构成的词向量矩阵
    return: 返回拼接而成的一维词向量
    '''
    # 最终的词向量
    result_list = []
    for window_size in range(2, 5):
        max_list, avg_list = [], []
        for k1 in range(len(embedding_matrix)):
            if int(k1+window_size) > len(embedding_matrix):
                break
            else:
                matrix01 = embedding_matrix[k1:k1+window_size]
                max_list.extend([np.max(matrix01)])  # 最大池化层
                avg_list.extend([np.mean(matrix01)])  # 均值池化层
        # 再将池化层和均值层拼接起来
        max_list.extend(avg_list)
        # 将窗口为2，3，4的embedding拼接起来
        result_list.extend(max_list)
    return result_list

该函数用于获取标签空间的词嵌入

In [10]:
def Find_Label_embedding(example_matrix, embedding):
    '''
    根据论文《Joint embedding of words and labels》获取标签空间的词嵌入
    parameters:
    -- example_matrix(np.array 2D): denotes the matrix of words embedding
    -- embedding(np.array 2D): denotes the embedding of all words in corpus
    return: (np.array 1D) the embedding by join label and word
    '''
    # 首先在预训练的词向量中获取标签的词向量句子,每一行表示一个标签表示
    # 每一行表示一个标签的embedding
    label_arr = np.array(
        [embedding.wv.get_vector(labelIndexToName[key])
         for key in labelIndexToName if labelIndexToName[key] in embedding.wv.vocab.keys()])

    # 根据consin来计算label与word之间的相似度,matrix01表示分子
    matrix01 = np.dot(label_arr, np.transpose(example_matrix))

    # 在计算consin的分母
    matrix02 = []
    for k1 in range(len(label_arr)):
        list01 = []
        for k2 in range(len(example_matrix)):
            list01.extend([np.linalg.norm(label_arr[k1]) *
                           np.linalg.norm(example_matrix[k2])])
        matrix02.append(list01)
    matrix02 = np.array(matrix02)
    # similarity表示通过consin相似度计算得到的矩阵
    similarity_matrix = matrix01/matrix02

    # 然后对相似矩阵进行均值池化，则得到了“类别-词语”的注意力机制
    # 这里可以使用max-pooling和mean-pooling
    attention = np.max(similarity_matrix, axis=0)
    attention_softmax = softmax(x=attention)
    # 将样本的词嵌入与注意力机制相乘得到
    attention_embedding = example_matrix * \
        attention_softmax.reshape(len(attention_softmax), 1)
    attention_embedding_avg = np.mean(attention_embedding, axis=0)
    attention_embedding_max = np.max(attention_embedding, axis=0)
    result_embedding = np.hstack(
        (attention_embedding_avg, attention_embedding_max))
    #print("label-word", result_embedding.shape)

    return result_embedding


联合多种特征工程来构造新的样本表示，
第一： 利用word-embedding的average pooling和max-pooling。
第二：分别利用窗口size=2，3，4对word-embedding进行卷积操作，然后再进行max/avg-pooling操作。
第三：利用类别标签的表示，增加了词语和标签之间的语义交互，以此达到对词级别语义信息更深层次的考虑。（Label-Embedding Attentive Model (LEAM)）

In [11]:
def sentence2vec(query):
    '''
    函数说明：联合多种特征工程来构造新的样本表示，主要通过以下三种特征工程方法
            第一：利用word-embedding的average pooling和max-pooling
            第二：利用窗口size=2，3，4对word-embedding进行卷积操作，然后再进行max/avg-pooling操作
            第二：利用类别标签的表示，增加了词语和标签之间的语义交互，以此达到对词级别语义信息更深层次的考虑
            另外，对于词向量超过预定义的长度则进行截断，小于则进行填充
    参数说明：
    - query:数据集中的每一个样本
    return: 返回样本经过哦特征工程之后得到的词向量
    '''
    global max_length
    arr = []
    # 加载fast_embedding,w2v_embedding
    global fast_embedding, w2v_embedding
    fast_arr = np.array([fast_embedding.wv.get_vector(s)
                         for s in query if s in fast_embedding.wv.vocab.keys()])
    # 在fast_arr下滑动获取到的词向量
    if len(fast_arr) > 0:
        windows_fastarr = np.array(Find_embedding_with_windows(fast_arr))
        result_attention_embedding = Find_Label_embedding(
            fast_arr, fast_embedding)
    else:# 如果样本中的词都不在字典，则该词向量初始化为0
        # 这里300表示训练词嵌入设置的维度为300
        windows_fastarr = np.zeros(300) 
        result_attention_embedding = np.zeros(300)

    fast_arr_max = np.max(np.array(fast_arr), axis=0) if len(
        fast_arr) > 0 else np.zeros(300)
    fast_arr_avg = np.mean(np.array(fast_arr), axis=0) if len(
        fast_arr) > 0 else np.zeros(300)

    fast_arr = np.hstack((fast_arr_avg, fast_arr_max))
    # 将多个embedding进行横向拼接
    arr = np.hstack((np.hstack((fast_arr, windows_fastarr)),
                     result_attention_embedding))
    global sentence_max_length
    # 如果样本的维度大于指定的长度则需要进行截取或者拼凑,
    result_arr = arr[:sentence_max_length] if len(arr) > sentence_max_length else np.hstack((
        arr, np.zeros(int(sentence_max_length-len(arr)))))
    return result_arr

特征选择/抽取函数，由于经过特征工程得到的样本表示维度很高，因此需要进行降维 max_length表示降维之后的样本最大的维度。这里通过PCA方法降维

In [12]:
def Dimension_Reduction(Train, Test):
    '''
    函数说明：该函数通过PCA算法对样本进行降维，由于目前维度不是特别搞高 ，可以选择不降维。
    参数说明：
    - Train: 表示训练数据集
    - Test: 表示测试数据集
    Return: 返回降维之后的数据样本
    '''
    global max_length
    pca = PCA(n_components=max_length)
    pca_train = pca.fit_transform(Train)
    pca_test = pca.fit_transform(Test)

    return pca_train, pca_test

生成训练集与测试集的词向量并进行归一化处理，获取样本经过特征工程之后的样本表示，

In [13]:
def Find_Embedding():
    '''
    函数说明：该函数用于获取经过特征工程之后的样本表示
    Return:训练集特征数组(2D)，测试集特征数组(2D)，训练集标签数组（1D）,测试集标签数组（1D）
    '''
    print("获取样本表示中...")
    min_max_scaler = preprocessing.MinMaxScaler()
    Train_features2 = min_max_scaler.fit_transform(
        np.vstack(train["queryCutRMStopWord"].apply(sentence2vec)))
    Test_features2 = min_max_scaler.fit_transform(
        np.vstack(test["queryCutRMStopWord"].apply(sentence2vec)))
    print("获取样本词表示完成")
    # 通过PCA对样本表示进行降维
    Train_features2, Test_features2 = Dimension_Reduction(
        Train=Train_features2, Test=Test_features2)
    Train_label2 = train["labelIndex"]
    Test_label2 = test["labelIndex"]

    print("加载训练好的词向量")
    print("Train_features.shape =", Train_features2.shape)
    print("Test_features.shape =", Test_features2.shape)
    print("Train_label.shape =", Train_label2.shape)
    print("Test_label.shape =", Test_label2.shape)

    return Train_features2, Test_features2, Train_label2, Test_label2


通过该函数输出模型在训练集和测试集上的准确率

In [14]:
def Predict(Train_label, Test_label, Train_predict_label, Test_predict_label, model_name):
    '''
    函数说明：直接输出训练集和测试在模型上的准确率
    参数说明：
        - Train_label: 真实的训练集标签（1D）
        - Test_labelb: 真实的测试集标签（1D）
        - Train_predict_label: 模型在训练集上的预测的标签(1D)
        - Test_predict_label: 模型在测试集上的预测标签（1D）
        - model_name: 表示训练好的模型
    Return: None
    '''
    # 输出训练集的准确率
    print(Search_Flag+model_name+'_'+'Train accuracy %s' % metrics.accuracy_score(
        Train_label, Train_predict_label))
    # 输出测试集的准确率
    print(Search_Flag+model_name+'_'+'test accuracy %s' % metrics.accuracy_score(
        Test_label, Test_predict_label))

根据Grid搜索方法来求模型最优的分类结果参数并保存训练好的模型

In [15]:
def Grid_Train_model(Train_features, Test_features, Train_label, Test_label):
    '''
    函数说明：基于网格搜索优化的方法搜索模型最优参数，最后保存训练好的模型
    参数说明：
        - Train_features: 训练集特征数组（2D）
        - Test_features: 测试集特征数组（2D）
        - Train_label: 真实的训练集标签 (1D)
        - Test_label: 真实的测试集标签（1D）
    Return: None
    '''
    parameters = {
        'max_depth': [5, 10, 15, 20, 25],
        'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15],
        'n_estimators': [100, 500, 1000, 1500, 2000],
        'min_child_weight': [0, 2, 5, 10, 20],
        'max_delta_step': [0, 0.2, 0.6, 1, 2],
        'subsample': [0.6, 0.7, 0.8, 0.85, 0.95],
        'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9],
        'reg_alpha': [0, 0.25, 0.5, 0.75, 1],
        'reg_lambda': [0.2, 0.4, 0.6, 0.8, 1],
        'scale_pos_weight': [0.2, 0.4, 0.6, 0.8, 1]

    }
    # 定义分类模型列表，这里仅使用LightGBM模型
    models = [
        lgb.LGBMClassifier(objective='multiclass', n_jobs=10, num_class=33, num_leaves=30, reg_alpha=10, reg_lambda=200,
                           max_depth=3, learning_rate=0.05, n_estimators=2000, bagging_freq=1, bagging_fraction=0.9, feature_fraction=0.8, seed=1440),
    ]
    # 遍历模型
    for model in models:
        model_name = model.__class__.  __name__
        gsearch = GridSearchCV(
            model, param_grid=parameters, scoring='accuracy', cv=3)
        gsearch.fit(Train_features, Train_label)
        # 输出最好的参数
        print("Best parameters set found on development set:{}".format(
            gsearch.best_params_))
        Test_predict_label = gsearch.predict(Test_features)
        Train_predict_label = gsearch.predict(Train_features)
        Predict(Train_label, Test_label,
                Train_predict_label, Test_predict_label, model_name)
    # 保存训练好的模型
    joblib.dump(model, Search_Flag+'_'+model_name+'.pkl')


主函数,先求训练集和测试集的词向量，然后根据Grid搜索来找到最佳参数的分类模型

In [ ]:
Train_features, Test_features, Train_label, Test_label = Find_Embedding()
Grid_Train_model(Train_features=Train_features, Test_features=Test_features,Train_label=Train_label, Test_label=Test_label)